# DF Cleaner

## Data cleaning

In [5]:
import pandas as pd

df = pd.read_excel(r"C:\Users\monaa\Dropbox\Thesis\Secundair\Metadata_OBV_gesplitst.xlsx")
len(df)

10992

In [271]:
#Delete trailing and leading whitespaces.
df['Afbeelding'] = df['Afbeelding'].str.strip() 
df['Collectie'] = df['Collectie'].str.strip() 
df['Productie'] = df['Productie'].str.strip() 
df['Gelegenheid'] = df['Gelegenheid'].str.strip() 


In [272]:
#New boolean column, which indicates whether the performance was part of a multiple bill (True) or not (False).

multiple_bills = []
for row in df['Productie']:
    if '+' in str(row):
        multiple_bills.append(True)
    else:
        multiple_bills.append(False)


df['Multiple bill'] = multiple_bills
df['Multiple bill'].value_counts()

False    10466
True       526
Name: Multiple bill, dtype: int64

In [273]:
#New column with a random ID voor each performance.

import random
df['Performances_ID'] = random.sample(range(100000, 999999), df.shape[0])


In [274]:
# Cells that include a '+' , i.e. a multiple bill are split so that each row only contains one wok.
# Through the Performance_ID we can keep track of those performances that were performed as a multiple bill.

df = df.assign(Productie=df.Productie.str.split("+")).explode('Productie')
df = df.reset_index(drop=True)
df.head()

,Afbeelding,Collectie,Productie,Datum,Gelegenheid,Multiple bill,Performances_ID
0,K.V.O. programmas 1893-97_00001-20211201_11330...,1893-1897,NaN,NaN,NaN,False,926614
1,K.V.O. programmas 1893-97_00002-20211201_11332...,1893-1897,NaN,NaN,NaN,False,510734
2,K.V.O. programmas 1893-97_00003-20211201_11333...,1893-1897,NaN,NaN,NaN,False,882184
3,K.V.O. programmas 1893-97_00004-20211201_11334...,1893-1897,NaN,NaN,NaN,False,433404
4,K.V.O. programmas 1893-97_00005-20211201_11335...,1893-1897,De Vrijschutter,3/10/1893,Eerste vertoning,False,868868


## Normalise spelling

In [275]:
#Establish dictionary in I keep track of spelling variations.
VariatieC = {
"Het Minnebrugje" : "T' Minnebrugje Het Minnebrugsken Het minnebrugje Minnebrugje",
"Het Arendsnest" : "Arendsnest",
"Bohème" : "Boheme",
"De Bruid der Zee" : "Bruid der Zee",
"Het Kind" : "Das Kind",
"Czaar en Timmerman" : "Csaar en Timmerman",
"De Verzonken Klok" : "De Verzonkene Klok",
"De Bedelstudent" : "De Arme student",
"De Doode Oogen" : "De Dode Ogen",
"De Doode Stad" : "De Dode Stad",
"De Koningin van Saba" : "De Koningin van Saaba",
"De Lustige Boer" : "De Lustige Broeder Lustige Boer",
"De Lustige Oorlog" : "De Vrolijke Oorlog",
"De Lustige Vrouwtjes van Windsor" : "The Merry Wives of Windsor De Vrolijke Vrouwtjes van Windsor Lustige Vrouwtjes van Windsor",
"De Meesterzangers van Nurenberg" : "De Meesterzanger De Meesterzangers De Meesterzangers van Nuremberg De Meesterzangers van Nürnberg", 
"De Tooverfluit" : "Tooverfluit Tooverfluit",
"De Tragedie van Salomé" : "De tragedie van Salomé",
"De Twee Medaillons" : "De Tweede Medaillons",
"De Vestale" : "Vestale",
"De Vrijschutter" : "Vrijschutter",
"De Walkure" : "De Walküre La Walkyrie Die Walkure",
"De Zeven Hoofdzonden" : "de Zeven Hoofdzonden",
"De Drie Houten Poppen" : "De drie Houten Poppen",
"De Evangelieman" : "Die Evangelieman",
"Don Juan" : "Don Giovanni",
"De Draaimolen" : "Draaimolen",
"Driemeisjeshuis" : "Het Driemeisjeshuis",
"Een Nacht te Grenada" : "Eeen Nacht te Grenada",
"Ein Walzertraum" : "Walzertaum",
"Hansje en Grietje" : "Hans en Grietje",
"Herbergprinses" : "Herbergrprinses Herberprinses",
"Het Rijngoud" : "Rijngoud",
"Het Rozemarijntje" : "Rosemarijntje Rozemarijntje",
"Winternachtsdroom" : "Het Winternachtsdroom",
"Zeevolk" : "Het Zeevolk",
"Hoffmann's Vertellingenen" : "Hoffmann's Vertellingen",
"Jean-Marie" : "Jose Maria Jean Marie Marie José",
"Jozef in Egypte" : "Josef in Egypte",
"De Kinderen der Zee": "Kinderen der Zee",
"Liefdetoover" : "Liefde Toover",
"Marieke van Nijmegem" : "Marieke van Nijmegen Marike van Nymegem",
"Orpheus' Dood" : "Orpheus's Dood",
"Paljas" : "Pagliacci",
"Parisina" : 'Parsina',
"Quinten Massijs" : "Quinten Massys Quinten Metsys Quinten-Massijs Quinten-Massys",
"Reeks van Oud-Grieksche Dansen" : "Reeks van Oud-Griekscge Dansen",
"Reinaert de Vos" : "Reinart de Vos",
"De Rijndwergen": "Rijndwergen",
"Tannhauser" : "Tannhäuser",
"Theroigne de Mericout" : "Theroigne De Mericourt Theroigne de Mericoux Théroigne de Méricourt Théroigne de Méricout",
"Toespraak van den Heer Dr. Maurits Sabbe, conservator van het Museum Plantijn" : "Toespraal van den Heer Dr. Maurits Sabbe, conservator van het Museum Plantijn",
"Tweede Dansenreeks" : "Tweede dansreeks",
"Zwaargewicht" : "Zwaargewicht ( of De eer van't Land)"
}

In [276]:
#create functions that iterates over the items in the dictionary and returns the key if the title is in the values.
#if it is not in the values, there was not spelling variation for this title and the title is returned.
def clean_titles (row):

    for key, values in VariatieC.items():
        if str(row) in str(values):
            #print('it is in the values')
            return key
        elif row == key:
            return key
    else:
        return row
    

In [277]:
#Test it on some variations.

a = 'Quinten Metsys'
b = "Tannhäuser"
c = 'Madame Butterfly'

print(clean_titles(a))
print(clean_titles(b))
print(clean_titles(c))


Quinten Massijs
Tannhauser
Madame Butterfly


In [278]:
#Function is now applied to the entire dataset. The normalised spelling is gathered in a new column called 'Genormaliseerde titel'.
df['Genormaliseerde titel'] = df['Productie'].apply(clean_titles)


## Add metadata

Load dictionary-formatted txt-file containing additional metadata.
- key = normalised title
- value [0] = original title
- value [1] = composer
- value [2] = year of first performance
- value [3] = original title
- value [4] = genre


In [3]:
with open('C:/Users/monaa/Dropbox/Thesis/secundair/META.txt','r', encoding = 'UTF8') as inf:
    Metadata = eval(inf.read())


In [4]:
#Check if the txt-file is formatted correctly
output = []
for key, values in Metadata.items():
    if len(values) != 5:
       output.append(values)
       print(output)

if len(output) == 0:
    print('Looking good!')

Looking good!


In [281]:
#Define a function that identifies the original lanagage based on the metadata.

def add_taal (row):
    for key, values in Metadata.items():
        if row == key:
            return values[0]
    else:
        return 'N/A'

#try it out.

a = "Madame Butterfly"
b = "Tannhauser"
c = "Quinten Massijs"

print(add_taal(a))
print(add_taal(b))
print(add_taal(c))


ITA
DUI
NL


In [282]:
#Define a function that identifies the composer based on the metadata.

def add_composer (row):
    for key, values in Metadata.items():
        if row == key:
            return values[1]
    else:
        return 'N/A'
    
#try it out.

print(add_composer(a))
print(add_composer(b))
print(add_composer(c))


Giacomo Puccini
Richard Wagner
Emile Wambach


In [283]:
#Define a function that identifies the year of the first performance based on the metadata.

def add_premiere (row):
    for key, values in Metadata.items():
        if row == key:
            return values[2]
    else:
        return 'N/A'

print(add_premiere(a))
print(add_premiere(b))
print(add_premiere(c))


1904
1845
1899


In [284]:
#Define a function that identifies the year of the original title based on the metadata.

def add_oortitel (row):
    for key, values in Metadata.items():
        if row == key:
            return values[3]
    else:
        return 'N/A'


print(add_oortitel(a))
print(add_oortitel(b))
print(add_oortitel(c))

Madama Butterfly
Tannhäuser
Quinten Massys


In [285]:
#Define a function that identifies the year of the genre based on the metadata.

def add_genre (row):
    for key, values in Metadata.items():
        if row == key:
            return values[4].lower()
    else:
        return 'N/A'


print(add_genre(a))
print(add_genre(b))
print(add_genre(c))

opera seria
romantische oper
lyrisch drama


In [286]:
#add the metadata as new columns in the dataframe.

df['Oorspronkelijke taal'] = df['Genormaliseerde titel'].apply(add_taal)
df['Componist'] = df['Genormaliseerde titel'].apply(add_composer)
df['Oorspronkelijke première'] = df['Genormaliseerde titel'].apply(add_premiere)
df['Oorspronkelijke titel'] = df['Genormaliseerde titel'].apply(add_oortitel)
df['genre'] = df['Genormaliseerde titel'].apply(add_genre)
df.sample(5)


,Afbeelding,Collectie,Productie,Datum,Gelegenheid,Multiple bill,Performances_ID,Genormaliseerde titel,Oorspronkelijke taal,Componist,Oorspronkelijke première,Oorspronkelijke titel,genre
6213,1925-1926_00164-20220208_094314.jpg,1925-1926,NaN,NaN,NaN,False,735633,NaN,N/A,N/A,N/A,N/A,N/A
9738,1930-1931_00357-20220209_163751.jpg,1930-1931,NaN,NaN,NaN,False,125070,NaN,N/A,N/A,N/A,N/A,N/A
897,1897-1900 Nederlandsch Lyrisch Tooneel. Progra...,1897-1900,Cleopatra,1900-03-04 00:00:00,Buitengewone vertoning inden Cirkschouwburg,False,102863,Cleopatra,DEE,August Enna,1894,Cleopatra,opera seria
5214,1923-1924_00183-20220207_153101.jpg,1923-1924,De Vrolijke Vrouwtjes van Windsor,1923-11-20 00:00:00,NaN,False,634330,De Lustige Vrouwtjes van Windsor,DUI,Otto Nicolai,1849,Die lustigen Weiber von Windsor,zangspel
9293,1929-1930 bis_00286-20220209_140705.jpg,1929-1930,NaN,NaN,NaN,False,437381,NaN,N/A,N/A,N/A,N/A,N/A


## Additional preprocessing

In [292]:
#de kolom 'Datum' wordt omgezet naar datetime type.

df['Datum'] = pd.to_datetime(df['Datum'], dayfirst=True, errors='coerce')
df.head()

,Afbeelding,Collectie,Productie,Datum,Gelegenheid,Multiple bill,Performances_ID,Genormaliseerde titel,Oorspronkelijke taal,Componist,Oorspronkelijke première,Oorspronkelijke titel,genre
0,K.V.O. programmas 1893-97_00001-20211201_11330...,1893-1897,NaN,NaT,NaN,False,926614,NaN,N/A,N/A,N/A,N/A,N/A
1,K.V.O. programmas 1893-97_00002-20211201_11332...,1893-1897,NaN,NaT,NaN,False,510734,NaN,N/A,N/A,N/A,N/A,N/A
2,K.V.O. programmas 1893-97_00003-20211201_11333...,1893-1897,NaN,NaT,NaN,False,882184,NaN,N/A,N/A,N/A,N/A,N/A
3,K.V.O. programmas 1893-97_00004-20211201_11334...,1893-1897,NaN,NaT,NaN,False,433404,NaN,N/A,N/A,N/A,N/A,N/A
4,K.V.O. programmas 1893-97_00005-20211201_11335...,1893-1897,De Vrijschutter,1893-10-03,Eerste vertoning,False,868868,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper


In [293]:
#Blanco pagina's of irrelevante pagina's worden verwijderd.

old_len = len(df)
df = df.dropna(subset=['Productie','Datum'])
print(f'{old_len - len(df)} of {old_len} records removed')
df.head()


6267 of 11600 records removed


,Afbeelding,Collectie,Productie,Datum,Gelegenheid,Multiple bill,Performances_ID,Genormaliseerde titel,Oorspronkelijke taal,Componist,Oorspronkelijke première,Oorspronkelijke titel,genre
4,K.V.O. programmas 1893-97_00005-20211201_11335...,1893-1897,De Vrijschutter,1893-10-03,Eerste vertoning,False,868868,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper
6,K.V.O. programmas 1893-97_00007-20211201_11341...,1893-1897,De Vrijschutter,1893-10-03,Eerste vertoning,False,361571,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper
8,K.V.O. programmas 1893-97_00009-20211201_11343...,1893-1897,Willem Tell,1893-10-05,Eerste vertoning,False,445427,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper
10,K.V.O. programmas 1893-97_00011-20211201_11345...,1893-1897,De Vrijschutter,1893-10-10,NaN,False,738479,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper
12,K.V.O. programmas 1893-97_00013-20211201_11351...,1893-1897,Willem Tell,1893-10-12,NaN,False,724410,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper


In [294]:
#Duplicates worden verwijderd i.e. de productie en de datum zijn hetzelfde.

all_dups = len(df)
df = df.drop_duplicates(subset=['Productie', 'Datum'])
print(f'{all_dups - len(df)} of {all_dups} records removed! {len(df)} records remain')
df.head()


863 of 5333 records removed! 4470 records remain


,Afbeelding,Collectie,Productie,Datum,Gelegenheid,Multiple bill,Performances_ID,Genormaliseerde titel,Oorspronkelijke taal,Componist,Oorspronkelijke première,Oorspronkelijke titel,genre
4,K.V.O. programmas 1893-97_00005-20211201_11335...,1893-1897,De Vrijschutter,1893-10-03,Eerste vertoning,False,868868,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper
8,K.V.O. programmas 1893-97_00009-20211201_11343...,1893-1897,Willem Tell,1893-10-05,Eerste vertoning,False,445427,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper
10,K.V.O. programmas 1893-97_00011-20211201_11345...,1893-1897,De Vrijschutter,1893-10-10,NaN,False,738479,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper
12,K.V.O. programmas 1893-97_00013-20211201_11351...,1893-1897,Willem Tell,1893-10-12,NaN,False,724410,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper
14,K.V.O. programmas 1893-97_00015-20211201_11352...,1893-1897,Charlotte Corday,1893-11-16,NaN,False,301877,Charlotte Corday,NL,Peter Benoit,?,Charlotte Corday,lyrisch drama


In [295]:
#df wordt opgeslagen.
#df.to_csv('C:/Users/monaa/Dropbox/Thesis/Primair/KVO_META.csv', encoding = 'utf-8')